In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
from datetime import datetime, date, timedelta, timezone
import pytz as tz
from pytz import timezone

In [2]:
def obtener_stats(func_stats):
    stats_dict = {}
    
    date = func_stats.find('div', attrs={'class':'info match-link'})
    if date:
        stats_dict['date'] = date.get('starttime').replace('T',' ').replace(':00+00:00','')
    else:
        stats_dict['date'] = '-'
 
    zerozero = func_stats.find('div', attrs={'class':'content-box draw'}).find('div', attrs={'class':'cont'})
    if zerozero:
        stats_dict['zerozero'] = zerozero.get_text('/',strip=True).replace('%','')[4:10].replace('<','')
    else:
        stats_dict['zerozero'] = '-' 
        
    homeRank = func_stats.find('div', attrs={'class':'position t1'})
    if homeRank:
        stats_dict['homeRank'] = homeRank.get_text()
    else:
        stats_dict['homeRank'] = '-'    
            
    homeTeam = func_stats.find('p', attrs={'itemprop':'homeTeam'})
    if homeTeam:
        stats_dict['homeTeam'] = homeTeam.get_text('/', strip=True)
    else:
        stats_dict['homeTeam'] = '-'

    homeVictory = func_stats.find('div', attrs={'class':'content-box t-1'}).find('div', attrs={'class':'poss-box'})
    if homeVictory:
        stats_dict['homeVictory'] = homeVictory.get_text('/', strip=True).replace('/Probabilidad gana/',' - ')[0:2].replace('.','').replace('%','')
    else:
        stats_dict['homeVictory'] = '-'     
        
    homeGoals = func_stats.find('div', attrs={'class':'content-box t-1'}).find('div', attrs={'class':'poss-box'})
    if homeGoals:
        stats_dict['homeGoals'] = homeGoals.get_text('/', strip=True).replace('/Probabilidad gana/',' - ').replace('/Goles esperados','')[6::].replace('-','').replace('%','')
    else:
        stats_dict['homeGoals'] = '-'   

    awayGoals = func_stats.find('div', attrs={'class':'content-box t-2'}).find('div', attrs={'class':'poss-box'})
    if awayGoals:
        stats_dict['awayGoals'] = awayGoals.get_text('/', strip=True).replace('/Probabilidad gana/',' - ').replace('/Goles esperados','')[6::].replace('-','').replace('%','')
    else:
        stats_dict['awayGoals'] = '-'   
        
    awayVictory = func_stats.find('div', attrs={'class':'content-box t-2'}).find('div', attrs={'class':'poss-box'})
    if awayVictory:
        stats_dict['awayVictory'] = awayVictory.get_text('/', strip=True).replace('/Probabilidad gana/',' - ')[0:2].replace('.','').replace('%','')
    else:
        stats_dict['awayVictory'] = '-'     

    awayTeam = func_stats.find('p', attrs={'itemprop':'awayTeam'})
    if awayTeam:
        stats_dict['awayTeam'] = awayTeam.get_text('/', strip=True)
    else:
        stats_dict['awayTeam'] = '-'     
     
    awayRank = func_stats.find('div', attrs={'class':'position t2'})
    if awayRank:
        stats_dict['awayRank'] = awayRank.get_text()
    else:
        stats_dict['awayRank'] = '-' 
        
    competition = func_stats.find('h3', attrs={'class':'competition'})
    if competition:
        stats_dict['competition'] = competition.get_text('/', strip=True)
    else:
        stats_dict['competition'] = '-'
   
    return stats_dict

In [3]:
df = pd.DataFrame()

In [4]:
lista_competiciones = [
    'https://es.besoccer.com/competicion/ligue_1',
    'https://es.besoccer.com/competicion/escocia',
    'https://es.besoccer.com/competicion/portugal',
    'https://es.besoccer.com/competicion/segunda_brasilena',
    'https://es.besoccer.com/competicion/japon',
    'https://es.besoccer.com/competicion/liga_republica_irlanda',
    'https://es.besoccer.com/competicion/premier',
    'https://es.besoccer.com/competicion/segunda',
    'https://es.besoccer.com/competicion/bundesliga_2',
    'https://es.besoccer.com/competicion/j_two_league',
    'https://es.besoccer.com/competicion/brasil',
    'https://es.besoccer.com/competicion/league_one',
    'https://es.besoccer.com/competicion/ligue_2',
    'https://es.besoccer.com/competicion/primera_division_argentina',
    'https://es.besoccer.com/competicion/first_division',
    'https://es.besoccer.com/competicion/vitalis',
    'https://es.besoccer.com/competicion/serie_b',
    'https://es.besoccer.com/competicion/suecia',
    'https://es.besoccer.com/competicion/polonia',
    'https://es.besoccer.com/competicion/resultados/1_league_polonia/2023',
    'https://es.besoccer.com/competicion/resultados/2_league_polonia/2023',
    'https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo1',
    'https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo2',
    'https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo3',
    'https://es.besoccer.com/competicion/resultados/3_league_polonia/2023/grupo4',
    'https://es.besoccer.com/competicion/corea',
    'https://es.besoccer.com/competicion/k_league_corea',
    'https://es.besoccer.com/competicion/national_league_corea',
    'https://es.besoccer.com/competicion/apertura_mexico',
    'https://es.besoccer.com/competicion/liga_de_expansion_mx_apertura',
    'https://es.besoccer.com/competicion/usa'
    'https://es.besoccer.com/competicion/japon',
    'https://es.besoccer.com/competicion/j_two_league',
    'https://es.besoccer.com/competicion/japon_league_3'
    ]

In [5]:
hoy = date.today()
manana = (hoy + timedelta(days=1)).strftime('%Y-%m-%d')
pasado = (hoy + timedelta(days=2)).strftime('%Y-%m-%d')
elotro = (hoy + timedelta(days=3)).strftime('%Y-%m-%d')

lista_diaria = []
url_diaria1 = f'https://es.besoccer.com/livescore/{manana}'
for i in range(1,5):
    url = url_diaria1+f'/{i}'
    lista_diaria.append(url)
url_diaria2 = f'https://es.besoccer.com/livescore/{pasado}'
for i in range(1,5):
    url = url_diaria2+f'/{i}'
    lista_diaria.append(url)
url_diaria3 = f'https://es.besoccer.com/livescore/{elotro}'
for i in range(1,5):
    url = url_diaria3+f'/{i}'
    lista_diaria.append(url)

#lista_diaria

In [6]:
lista_scrapear = lista_competiciones + lista_diaria
lista_scrapear

['https://es.besoccer.com/competicion/ligue_1',
 'https://es.besoccer.com/competicion/escocia',
 'https://es.besoccer.com/competicion/portugal',
 'https://es.besoccer.com/competicion/segunda_brasilena',
 'https://es.besoccer.com/competicion/japon',
 'https://es.besoccer.com/competicion/liga_republica_irlanda',
 'https://es.besoccer.com/competicion/premier',
 'https://es.besoccer.com/competicion/segunda',
 'https://es.besoccer.com/competicion/bundesliga_2',
 'https://es.besoccer.com/competicion/j_two_league',
 'https://es.besoccer.com/competicion/brasil',
 'https://es.besoccer.com/competicion/league_one',
 'https://es.besoccer.com/competicion/ligue_2',
 'https://es.besoccer.com/competicion/primera_division_argentina',
 'https://es.besoccer.com/competicion/first_division',
 'https://es.besoccer.com/competicion/vitalis',
 'https://es.besoccer.com/competicion/serie_b',
 'https://es.besoccer.com/competicion/suecia',
 'https://es.besoccer.com/competicion/polonia',
 'https://es.besoccer.com/c

In [7]:
for url in lista_scrapear:
    besoccer = requests.get(url)
    s_besoccer = BeautifulSoup(besoccer.text, 'lxml')
    try:
        matches = s_besoccer.find('div', attrs={'class':'comp-matches'}).find_all('a', attrs={'data-status':'-1'})
    except Exception as e1:
        print(i,'.- La url ',url,' no se pudo scrapear')
        print(i,'.-',e1)
        print(i,'.- Probando otra configuración...')
        try:
            matches = s_besoccer.find('div', attrs={'class':'matches'}).find_all('a', attrs={'data-status':'-1'})
        except Exception as e2:
            print(i,'.- La url ',url,' no se pudo scrapear')
            print(i,'.-',e2)

    links_matches = [match.get('href') for match in matches]
    for i in range(len(links_matches)):
        list_analisis = []
        r_match = requests.get(links_matches[i])
        s_match = BeautifulSoup(r_match.text, 'lxml')
        try:
            find_analisis = s_match.find('div', attrs={'class':'menu-scroll'}).select('a[href*="analisis"]')
            analisis = find_analisis[0]
            link_analisis = analisis.get('href')
            r_link_analisis = requests.get(link_analisis)
            s_link_analisis = BeautifulSoup(r_link_analisis.text, 'lxml')
            list_iter = [obtener_stats(s_link_analisis)]
            df_iter = pd.DataFrame(list_iter)
            df = pd.concat([df, df_iter], axis = 0, ignore_index = True)
            print(i,'.- Scrapeando',url)
            print(i,'.-', link_analisis)
        except Exception as e3:
            print(i,'.- La url ',links_matches[i],' no tiene sección análisis')
            print(i,'.-',e3)
            


0 .- Scrapeando https://es.besoccer.com/competicion/escocia
0 .- https://es.besoccer.com/partido/hibernian-fc/st-johnstone-fc/202323215/analisis
1 .- Scrapeando https://es.besoccer.com/competicion/escocia
1 .- https://es.besoccer.com/partido/heart-of-midlo/celtic-fc/202323214/analisis
2 .- Scrapeando https://es.besoccer.com/competicion/escocia
2 .- https://es.besoccer.com/partido/kilmarnock-fc/ross-county-fc/202323216/analisis
3 .- Scrapeando https://es.besoccer.com/competicion/escocia
3 .- https://es.besoccer.com/partido/motherwell/aberdeen/202323217/analisis
4 .- Scrapeando https://es.besoccer.com/competicion/escocia
4 .- https://es.besoccer.com/partido/glasgow-rangers-fc/livingston-fc/202323218/analisis
5 .- Scrapeando https://es.besoccer.com/competicion/escocia
5 .- https://es.besoccer.com/partido/st-mirren/dundee-united-fc/202323219/analisis
0 .- Scrapeando https://es.besoccer.com/competicion/portugal
0 .- https://es.besoccer.com/partido/fc-porto/benfica/202396087/analisis
1 .- Sc

9 .- Scrapeando https://es.besoccer.com/competicion/league_one
9 .- https://es.besoccer.com/partido/morecambe/barnsley-fc/202346369/analisis
10 .- Scrapeando https://es.besoccer.com/competicion/league_one
10 .- https://es.besoccer.com/partido/oxford-united/peterborough-u/202346370/analisis
11 .- Scrapeando https://es.besoccer.com/competicion/league_one
11 .- https://es.besoccer.com/partido/shrewsbury-town/charlton-athletic-fc/202346371/analisis
0 .- Scrapeando https://es.besoccer.com/competicion/ligue_2
0 .- https://es.besoccer.com/partido/amiens-sc/saint-etienne/202326020/analisis
1 .- Scrapeando https://es.besoccer.com/competicion/ligue_2
1 .- https://es.besoccer.com/partido/caen/rodez/202326021/analisis
2 .- Scrapeando https://es.besoccer.com/competicion/ligue_2
2 .- https://es.besoccer.com/partido/grenoble-foot-38/valenciennes-afc/202326022/analisis
3 .- Scrapeando https://es.besoccer.com/competicion/ligue_2
3 .- https://es.besoccer.com/partido/guingamp/bastia/202326023/analisis
4 

2 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/1
2 .- https://es.besoccer.com/partido/newcastle-united-fc/everton-fc/202318335/analisis
3 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/1
3 .- https://es.besoccer.com/partido/afc-bournemouth/southampton-fc/202318298/analisis
4 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/1
4 .- https://es.besoccer.com/partido/manchester-united-fc/tottenham-hotspur-fc/202318342/analisis
5 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/1
5 .- https://es.besoccer.com/partido/cadiz/betis/202349487/analisis
6 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/1
6 .- https://es.besoccer.com/partido/valladolid/celta/202349492/analisis
7 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/1
7 .- https://es.besoccer.com/partido/real-sociedad/mallorca/202349495/analisis
8 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/1
8 .- https://es.besoccer.com/partido/elche/real-madrid/202349465/an

7 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/2
7 .- https://es.besoccer.com/partido/usv-hercules/nac-breda/2023481671/analisis
8 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/2
8 .- https://es.besoccer.com/partido/ado-20/emmen/2023481680/analisis
9 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/2
9 .- https://es.besoccer.com/partido/spakenburg/sv-tielse-eendracht-combinatie/2023481667/analisis
10 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/2
10 .- https://es.besoccer.com/partido/nec/fortuna-sittard/2023481685/analisis
11 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/2
11 .- https://es.besoccer.com/partido/gazovik-orenburg/terek-groznyi/2023367262/analisis
12 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/2
12 .- https://es.besoccer.com/partido/fakel/spartak-moskva/2023367259/analisis
13 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/2
13 .- https://es.besoccer.com/partido/fc-olimpiyets-nizhni-n

11 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/3
11 .- https://es.besoccer.com/partido/bohemians-praga/trinec/2023482111/analisis
12 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/3
12 .- https://es.besoccer.com/partido/dukla-praha/slavia-praha/2023482122/analisis
13 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/3
13 .- https://es.besoccer.com/partido/inter-turku/vps-vaasa/2022672093/analisis
14 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/3
14 .- https://es.besoccer.com/partido/seinajoen-jk/oulu/2022672094/analisis
15 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/3
15 .- https://es.besoccer.com/partido/cf-talavera-reina/merida-ad-senior/2023161497/analisis
16 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/3
16 .- https://es.besoccer.com/partido/zorka-bdu/belshina/2022398893/analisis
17 .- Scrapeando https://es.besoccer.com/livescore/2022-10-19/3
17 .- https://es.besoccer.com/partido/fk-slutsk-363/dinamo-min

...

30 .- Scrapeando https://es.besoccer.com/livescore/2022-10-20/3
30 .- https://es.besoccer.com/partido/al-sinaah/newroz-sc/2023503792/analisis
31 .- Scrapeando https://es.besoccer.com/livescore/2022-10-20/3
31 .- https://es.besoccer.com/partido/naft-al-basra-sc/al-qasim/2023503793/analisis
32 .- Scrapeando https://es.besoccer.com/livescore/2022-10-20/3
32 .- https://es.besoccer.com/partido/al-kahrabaa/al-talaba/2023503795/analisis
33 .- Scrapeando https://es.besoccer.com/livescore/2022-10-20/3
33 .- https://es.besoccer.com/partido/al-shorta/al-diwaniya/2023503796/analisis
34 .- Scrapeando https://es.besoccer.com/livescore/2022-10-20/3
34 .- https://es.besoccer.com/partido/saint-eloi-lupopo/lubumbashi-sport/2023504048/analisis
35 .- Scrapeando https://es.besoccer.com/livescore/2022-10-20/3
35 .- https://es.besoccer.com/partido/motema-pembe/as-kuya-sport/2023504049/analisis
36 .- Scrapeando https://es.besoccer.com/livescore/2022-10-20/3
36 .- https://es.besoccer.com/partido/energetyk-bgu-

6 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/1
6 .- https://es.besoccer.com/partido/granada/real-zaragoza/202349784/analisis
7 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/1
7 .- https://es.besoccer.com/partido/philadelphia-union/fc-cincinnati/2022672354/analisis
8 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/1
8 .- https://es.besoccer.com/partido/los-angeles-football-club/angeles-galaxy/2022672355/analisis
9 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/1
9 .- https://es.besoccer.com/partido/hibernian-fc/st-johnstone-fc/202323215/analisis
10 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/1
10 .- https://es.besoccer.com/partido/antalyaspor/istanbulspor-as/202397702/analisis
11 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/1
11 .- https://es.besoccer.com/partido/hansa-rostock/kaiserslautern/202324481/analisis
12 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/1
12 .- https://es.besoccer.com/part

10 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/2
10 .- https://es.besoccer.com/partido/balingen/bahlinger-sc/2023115302/analisis
11 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/2
11 .- https://es.besoccer.com/partido/augsburg-ii/ansbach/202362075/analisis
12 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/2
12 .- https://es.besoccer.com/partido/alemannia-aachen/koln-ii/2023115607/analisis
13 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/2
13 .- https://es.besoccer.com/partido/rot-weib-erfurt/tennis-borussia/2023114971/analisis
14 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/2
14 .- https://es.besoccer.com/partido/bsv-schwarz-weiss-rehden/havelse/2023190324/analisis
15 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/2
15 .- https://es.besoccer.com/partido/al-wehda/al-sharjah/2022460541/analisis
16 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/2
16 .- https://es.besoccer.com/partido/sporting-club-c

14 .- La url  https://es.besoccer.com/partido/frome-town/westbury-united/2023253237  no tiene sección análisis
14 .- list index out of range
15 .- La url  https://es.besoccer.com/partido/norwich-city-sub21/west-bromwich-albion-sub-21/2023253735  no tiene sección análisis
15 .- list index out of range
16 .- La url  https://es.besoccer.com/partido/aston-villa-sub21/nottingham-forest-academy/2023253736  no tiene sección análisis
16 .- list index out of range
17 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/3
17 .- https://es.besoccer.com/partido/al-jazeera-314/al-hussein-sc/2022574003/analisis
18 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/3
18 .- https://es.besoccer.com/partido/moghayer-al-sarhan/al-faisaly-314/2022574004/analisis
19 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/3
19 .- https://es.besoccer.com/partido/expreso-rojo/atletico-huila/2022670753/analisis
20 .- Scrapeando https://es.besoccer.com/livescore/2022-10-21/3
20 .- https://es.be

18 .- La url  https://es.besoccer.com/partido/club-atletico-granadilla/union-arafo-fund-edgar-mendez/2023477118  no tiene sección análisis
18 .- list index out of range
19 .- La url  https://es.besoccer.com/partido/au-guimar-b/atletico-chene/2023477120  no tiene sección análisis
19 .- list index out of range
20 .- La url  https://es.besoccer.com/partido/sd-valleseco/cd-union-isora/2023477121  no tiene sección análisis
20 .- list index out of range
21 .- La url  https://es.besoccer.com/partido/ud-rambla/emf-matanza-b/2023474880  no tiene sección análisis
21 .- list index out of range
22 .- La url  https://es.besoccer.com/partido/sd-ravelo/realejos/2023474881  no tiene sección análisis
22 .- list index out of range
23 .- La url  https://es.besoccer.com/partido/guargacho/charco-pino/2023477117  no tiene sección análisis
23 .- list index out of range
24 .- La url  https://es.besoccer.com/partido/emf-santiago-del-teide/ud-taco-san-luis/2023477119  no tiene sección análisis
24 .- list index 

In [8]:
# cuenta el número de registros encontrados
totalPartidosScrapeados = len(df)
print(f'Número de partidos scrapeados: {totalPartidosScrapeados}')

Número de partidos scrapeados: 594


In [9]:
print('Quitar los partidos cuyo porcentaje de empatar es menor que 7%')
print('- ' * 10)
df['zerozero'] = df['zerozero'].astype('float')
df = df.loc[df['zerozero'] >= 7, :]
partidosMenor7 = totalPartidosScrapeados - len(df)
print(f'Se han eliminado {partidosMenor7} partidos')

Quitar los partidos cuyo porcentaje de empatar es menor que 7%
- - - - - - - - - - 
Se han eliminado 350 partidos


In [10]:
print('Buscar partidos duplicados')
print('- ' * 10)

# elimina los registros duplicados
df = df.drop_duplicates()

partidosDuplicados = totalPartidosScrapeados - partidosMenor7 - len(df)

# cuenta el número de registros restantes después de eliminar duplicados
print(f'Se han eliminado {partidosDuplicados} partidos')
print('- ' * 10)
print(f'Número de registros después de haber eliminado los partidos duplicados: {len(df)}')

Buscar partidos duplicados
- - - - - - - - - - 
Se han eliminado 19 partidos
- - - - - - - - - - 
Número de registros después de haber eliminado los partidos duplicados: 225


In [11]:
df['draw'] = 100 - df['homeVictory'].astype(int) - df['awayVictory'].astype(int)

In [12]:
print('Ajustar horario de los partidos')
print('- ' * 10)
hora = '%H'
ahora = datetime.now().strftime(hora)
besoccerzone = timezone('Etc/Greenwich')
besoccertime = datetime.now(besoccerzone).strftime(hora)
diferencia_horaria = int(ahora) - int(besoccertime)
df['date'] = pd.to_datetime(df["date"]) + pd.Timedelta(hours=diferencia_horaria)
print(f'Ajustado el horario de los partidos, {diferencia_horaria} horas según {besoccerzone}')

Ajustar horario de los partidos
- - - - - - - - - - 
Ajustado el horario de los partidos, 3 horas según Etc/Greenwich


In [13]:
print('Reordenar las columnas')
print('- ' * 10)
df = df.reindex(columns=['date','zerozero','homeRank','homeTeam','homeGoals','homeVictory','draw','awayVictory','awayGoals','awayTeam','awayRank','competition'])   

Reordenar las columnas
- - - - - - - - - - 


In [14]:
df = df.round(decimals = 3)

In [15]:
dict_reemplazar = {
    'originalTeam': ['Alavés','América de Cali','Atl.Tucumán','Atlético','Avântul Reghin','CD Águila','CD Lugo','CD Olmedo',
'Cobán Imperial','Colón','Djurgårdens IF','FC St. Pauli','Ferencvárosi','Gloria Buzău','Guaraní','IFK Malmö',
'İstanbul Başakşehir','Jönköpings Södra','Lanús','Leganés','LKS Lódz','Mladost GAT','Mladost Lučani','Mirandés','Nacional Potosí',
'Patriotas Boyacá','Qarabağ','Raya2 Expansión','Real Betis','Águilas Doradas','Deportes Tolima','HB Chelghoum Laid',
'At. Nacional','Estudiantes La Plata','CA Huracán','V-Varen Nagasaki','Helsingborgs IF','Gimnàstic Tarragona','SD Amorebieta',
'CF La Nucía','Standard de Liège','Hull City','Viktoria Plzeň','Málaga','Jaguares FC','IFK Göteborg','Malmö FF'],
    'betTeam': ['Alaves','America de Cali','Atl Tucuman','Atletico','Avantul Reghin','CD Aguila','Lugo','Olmedo',
'Coban Imperial','Colon','Djurgardens IF','St Pauli','Ferencvarosi','Gloria Buzau','Guarani','Malmo IF',
'Basaksehir','Jonkopings Sodra','Lanus','Leganes','LKS Lodz','Mladost GAT Novi Sad','Mladost Lucani','Mirandes','Nacional Potosi',
'Boyaca Patriotas','Qarabag','Raya2','Betis','Aguilas Doradas Rionegro','Tolima','Chelghoum Laid','Atletico Nacional Medellin',
'Estudiantes','Huracan','Nagasaki','Helsingborgs','Gimnastic','Amorebieta','Nucia','Standard Liege','Hull','Plzen','Malaga',
'Jaguares de Cordoba','IFK Goteborg','Malmo FF']
}
df_reemplazar = pd.DataFrame(dict_reemplazar)
df_reemplazar.sort_values(['betTeam'])

,originalTeam,betTeam
29,Águilas Doradas,Aguilas Doradas Rionegro
0,Alavés,Alaves
1,América de Cali,America de Cali
38,SD Amorebieta,Amorebieta
2,Atl.Tucumán,Atl Tucuman
3,Atlético,Atletico
32,At. Nacional,Atletico Nacional Medellin
4,Avântul Reghin,Avantul Reghin
16,İstanbul Başakşehir,Basaksehir
28,Real Betis,Betis


In [16]:
print('Renombrando equipos de columna home')
print('- ' * 10) 
for item in df.index:
    a = df.homeTeam[item]
    if a in df_reemplazar.values:
        posicionHome = df_reemplazar[df_reemplazar['originalTeam'] == a].index.tolist()
        for e in posicionHome:
            newTeam = df_reemplazar.betTeam[e]
            df = df.replace({a}, {newTeam}, regex=False)
            print(f'Equipo renombrado: {a} --> {newTeam}')
    else:
        pass 

Renombrando equipos de columna home
- - - - - - - - - - 
Equipo renombrado: Alavés --> Alaves
Equipo renombrado: CD Lugo --> Lugo
Equipo renombrado: Leganés --> Leganes
Equipo renombrado: Colón --> Colon
Equipo renombrado: Lanús --> Lanus
Equipo renombrado: Malmö FF --> Malmo FF
Equipo renombrado: Nacional Potosí --> Nacional Potosi


In [17]:
print('Renombrando equipos de columna away')
print('- ' * 10) 
for item in df.index:
    a = df.awayTeam[item]
    if a in df_reemplazar.values:
        posicionAway = df_reemplazar[df_reemplazar['originalTeam'] == a].index.tolist()
        for e in posicionAway:
            newTeam = df_reemplazar.betTeam[e]
            df = df.replace({a}, {newTeam}, regex=False)
            print(f'Equipo renombrado: {a} --> {newTeam}')
    else:
        pass

Renombrando equipos de columna away
- - - - - - - - - - 
Equipo renombrado: Guaraní --> Guarani
Equipo renombrado: Málaga --> Malaga
Equipo renombrado: Helsingborgs IF --> Helsingborgs
Equipo renombrado: Djurgårdens IF --> Djurgardens IF
Equipo renombrado: Real Betis --> Betis
Equipo renombrado: Hull City --> Hull
Equipo renombrado: Standard de Liège --> Standard Liege
Equipo renombrado: CD Olmedo --> Olmedo


In [18]:
dict_eliminar = {
    'borrarLiga': [
        'Botola Pro',
        'Rusia', 
        'Rusa',
        'Uefa Youth League',
        'CONCACAF',
        'Segunda Sudáfrica',
        'Segunda Indonesia',
        'Tercera Georgia',
        'Liga Somalia',
        'Liga Ruanda'
    ]
}

df_borrarLiga = pd.DataFrame(dict_eliminar)

In [19]:
# elimina equipos de ligas no requeridas
print('Eliminar partidos de ligas no deseadas')
print('- ' * 10) 
print(f'Número de registros actualmente: {len(df)}')
print('- ' * 10)        
for item in df_borrarLiga.index:
    a = df_borrarLiga.borrarLiga[item]
    df.drop(df[df['competition'].str.contains(a)].index, inplace=True)
    print(f'Liga {a}')
else:
    pass
        
print('- ' * 10)        
print(f'Número de registros después de eliminar las ligas no deseadas: {len(df)}')

Eliminar partidos de ligas no deseadas
- - - - - - - - - - 
Número de registros actualmente: 225
- - - - - - - - - - 
Liga Botola Pro
Liga Rusia
Liga Uefa Youth League
Liga CONCACAF
Liga Segunda Sudáfrica
Liga Segunda Indonesia
Liga Tercera Georgia
Liga Liga Somalia
Liga Liga Ruanda
- - - - - - - - - - 
Número de registros después de eliminar las ligas no deseadas: 217


In [20]:
df.style.set_properties(**{'text-align': 'center'})

,date,zerozero,homeRank,homeTeam,homeGoals,homeVictory,draw,awayVictory,awayGoals,awayTeam,awayRank,competition
0,2022-10-21 21:30:00,9.900000,3º,Hibernian FC,1.38,47,28,25,0.93,St. Johnstone,9º,"Liga Escocia. Temporada Regular, Jornada 12"
2,2022-10-22 17:00:00,7.700000,12º,Kilmarnock,1.71,57,25,18,0.86,Ross County FC,10º,"Liga Escocia. Temporada Regular, Jornada 12"
3,2022-10-22 17:00:00,13.000000,8º,Motherwell,1.14,41,31,28,0.9,Aberdeen,4º,"Liga Escocia. Temporada Regular, Jornada 12"
5,2022-10-22 17:00:00,12.700000,5º,St. Mirren,1.19,43,30,27,0.87,Dundee United,11º,"Liga Escocia. Temporada Regular, Jornada 12"
7,2022-10-22 17:30:00,15.700000,15º,Famalicão,1.28,54,30,16,0.57,Paços de Ferreira,17º,Liga Portuguesa. Jornada 10
10,2022-10-23 17:30:00,16.900000,18º,Marítimo,1.04,41,34,25,0.74,Arouca,11º,Liga Portuguesa. Jornada 10
11,2022-10-23 17:30:00,11.800000,12º,Vizela,1.11,37,31,32,1.02,CD Santa Clara,16º,Liga Portuguesa. Jornada 10
14,2022-10-24 22:15:00,14.600000,13º,Rio Ave,1.19,47,31,22,0.73,Portimonense,7º,Liga Portuguesa. Jornada 10
15,2022-10-19 01:00:00,22.200000,19º,Brusque,0.75,31,38,31,0.76,Novorizontino,15º,Serie B - Brasil. Jornada 36
16,2022-10-19 03:30:00,8.500000,1º,Cruzeiro,1.72,61,24,15,0.74,Guarani,14º,Serie B - Brasil. Jornada 36


In [21]:
# ordena un DataFrame por columna
# se puede ordenar por varias columnas añadiendo el nombre de la siguiente columna e.j. ('date', 'competition')

df.sort_values(['date', 'homeTeam']).to_csv('matches.csv', header = True)

In [22]:
################################################################################################
################################################################################################
## REVISIÓN DE VERSIONES                                                                      ##
##                                                                                            ##
## v.3.2.                                                                                     ##
##  - añadido filtro para cambiar nombres de equipos                                          ##
##  - añadido comando para eliminar ciertos registros segúna las ligas que no interesan       ##
##                                                                                            ##
## v.3.1.                                                                                     ##
## - Menú para elegir si scrapear sólo competiciones, sólo partidos diarios o todo (VS Code)  ##
## - limpieza de código                                                                       ##
## - sustituido .append por pd.concat                                                         ##
## - detecta automáticamente los tres días siguientes para escrapear partidos diarios         ##
## - detecta automáticamente zona horaria para ajustar hora de los partidos                   ##
##                                                                                            ##
## v.3.0                                                                                      ##
## - hace scraping de uns lista de competiciones y página de partidos diarios                 ##
## - busca registros duplicados y los elimina                                                 ##
## - añade columna con los porcentajes de partidos NoGoal                                     ##
##                                                                                            ##
## v.2.1                                                                                      ##
## - añade la paginación de los partidos diarios                                              ##
##                                                                                            ##
## v.2.0                                                                                      ##
## - scraping de una página específica de partidos diarios                                    ##
## - crea dataframe y lo envía a archivo .csv                                                 ##
##                                                                                            ##
################################################################################################
################################################################################################

In [23]:
# se reemplaza el nombre de algunos equipos para ser compatibles con las plataformas
# df = df.replace("Credicoop San Cristóbal", "Credicoop San Cristobal")

In [24]:
# se eliminan registros que incluyan competiciones que no interesan
# df.drop(df[df['competition'].str.contains('Apertura')].index, inplace=True)

In [25]:
######################################################################
## métodos para trabajar con los datos de pandas                    ##
##                                                                  ##
## del(df['newcolumn'])  --- elimina una columna                    ##
## df.rename(columns = {'NoGoal':'NoGoal%'}, inplace = True)        ##
######################################################################

In [26]:
# df.loc[df.competition.str.contains('liga escocia', case = False), "NoGoal%"] = '92.85'
# df.loc[df.competition.str.contains('serie b - brasil', case = False), "NoGoal%"] = '92.06'
# df.loc[df.competition.str.contains('liga fútbol japón', case = False), "NoGoal%"] = '88.67'
# df.loc[df.competition.str.contains('liga irlanda.', case = False), "NoGoal%"] = '87.67'
# df.loc[df.competition.str.contains('segunda división.', case = False), "NoGoal%"] = '86.36'
# df.loc[df.competition.str.contains('segunda japonesa j2', case = False), "NoGoal%"] = '84.72'
# df.loc[df.competition.str.contains('league one.', case = False), "NoGoal%"] = '84.25'
# df.loc[df.competition.str.contains('liga profesional argentina', case = False), "NoGoal%"] = '83.45'
# df.loc[df.competition.str.contains('ligue 2', case = False), "NoGoal%"] = '83.14'